In [1]:
import geopandas as gpd
import rasterio
from rasterio.mask import mask

In [2]:
# Cargar el shapefile que contiene el polígono para el recorte
shapefile_path = r'C:\Test\LOTE_CAMPODULCE.shp'
poligono = gpd.read_file(shapefile_path)

In [4]:
# Cargar el raster (NDVI)
raster_path = r'C:\Users\bismarksr\Downloads\30_CAMPO_DULCE_NDVI_2024-01-27.tif'
with rasterio.open(raster_path) as src:
    # El polígono debe estar en formato GeoJSON para la función mask
    geometrias = [poligono.geometry.values[0].__geo_interface__]
    
    # Crear la máscara
    out_image, out_transform = mask(src, geometrias, crop=True)
    
    # Copiar los metadatos del raster original para el nuevo archivo
    out_meta = src.meta.copy()

In [6]:
# Actualizar los metadatos para reflejar las nuevas dimensiones y transformación
out_meta.update({"driver": "GTiff",
                 "height": out_image.shape[1],
                 "width": out_image.shape[2],
                 "transform": out_transform})

In [8]:
# Guardar el raster recortado en un nuevo archivo
output_path = r'C:\Test\res.tif'
with rasterio.open(output_path, "w", **out_meta) as dest:
    dest.write(out_image)